In [4]:
from fmp_python.fmp import FMP
import os
import pandas as pd
from SP500_div_yield_crawler import SP500
from datetime import datetime

os.environ["FMP_API_KEY"] = "606d643d87241cde956b5cd85a3c56d1"
fmp = FMP()

# dividend, date  - historical dividend values and dates
dividends = fmp.get_dividends('O', 'historical-price-full/stock_dividend')

# set proper DateTime object as the index of the dataframe
dividends['Datetime'] = pd.to_datetime(dividends['date'])
dividends = dividends.set_index('Datetime')

In [5]:


# resample annually and sum dividend values. Note that year end (Dec 31) will be shown for all groups
# dividends = dividends.resample("A")["adjDividend"].sum()
# dividends.dtypes

dividends_processed = dividends.resample("A")["date"].count()
today = datetime.today()
this_year = today.year
mask = (dividends_processed.index > (str(this_year-1)+'-01-01')) & (dividends_processed.index <= (str(this_year-1)+'-12-31'))
filtered_dividends=dividends_processed.loc[mask]
# print(filtered_df)
# include = dividends[dividends['Date'].datetime.year == this_year]

In [6]:

year = 2021
mask = (dividends.index > (str(year)+'-01-01')) & (dividends.index <= (str(year)+'-12-31'))
dividends_filtered_of_year=dividends.loc[mask]

list_of_div_months = dividends_filtered_of_year.index.tolist()
list_of_div_months = [date.strftime("%m") for date in list_of_div_months]
print (list_of_div_months)


['01']


In [ ]:
short = fmp.get_quote_short('KO')
